In [338]:
# Dpendencies
from splinter import Browser
from bs4 import BeautifulSoup
import datetime

In [288]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

/usr/local/bin/chromedriver


In [289]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [300]:
base_url = 'https://www.debates.org'
url = 'https://www.debates.org/voter-education/debate-transcripts/'
# url = 'https://www.debates.org/voter-education/debate-transcripts/september-26-2016-debate-transcript/'
browser.visit(url)

In [301]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')


In [302]:
# for main page

results = soup.find_all('div', id='content-sm')
results[0]
# results


<div id="content-sm">
<h1>Debate Transcripts</h1>
<p><span class="graytext"><span class="grayheading">Unofficial transcripts of most presidential and vice presidential debates are available on this site.</span></span></p>
<p> </p>
<hr/>
<p class="graytext">2016 Transcripts</p>
<blockquote><p><a href="/voter-education/debate-transcripts/september-26-2016-debate-transcript/">September 26, 2016: The First Clinton-Trump Presidential Debate</a></p>
<p><a href="/voter-education/debate-transcripts/october-4-2016-debate-transcript/">October 4, 2016: The Kain-Pence Vice Presidential Debate</a></p>
<p><a href="/voter-education/debate-transcripts/october-9-2016-debate-transcript/">October 9, 2016: The Second Clinton-Trump Presidential Debate</a></p>
<p><a href="/voter-education/debate-transcripts/october-19-2016-debate-transcript/">October 19, 2016: The Third Clinton-Trump Presidential Debate</a></p></blockquote>
<hr/>
<p class="graytext">2012 Transcripts</p>
<blockquote><p><a href="/voter-educat

In [303]:
# for page 1 pulled text


# results = soup.find_all('div', id='content-sm')
# results
results = soup.find_all('div', id='content-sm')

# for i in range(0, len(results[0])):
for para in results[0].find_all('p', recursive='false'):
    print(para.text)
    
#           .getText())
#     text = [e.get_text() for e in results[0]]
#     print(text[0])
    

Unofficial transcripts of most presidential and vice presidential debates are available on this site.
 
2016 Transcripts
September 26, 2016: The First Clinton-Trump Presidential Debate
October 4, 2016: The Kain-Pence Vice Presidential Debate
October 9, 2016: The Second Clinton-Trump Presidential Debate
October 19, 2016: The Third Clinton-Trump Presidential Debate
2012 Transcripts
October 3, 2012: The First Obama-Romney Presidential Debate
October 11, 2012: The Biden-Ryan Vice Presidential Debate
October 16, 2012: The Second Obama-Romney Presidential Debate
October 22, 2012: The Third Obama-Romney Presidential Debate
2008 Transcripts
September 26, 2008: The First McCain-Obama Presidential Debate
October 2, 2008: The Biden-Palin Vice Presidential Debate
October 7, 2008: The Second McCain-Obama Presidential Debate
October 15, 2008: The Third McCain-Obama Presidential Debate
2004 Transcripts
October 13, 2004: The Third Bush-Kerry Presidential Debate
October 8, 2004: The Second Bush-Kerry P

In [304]:
len(results[0])

81

In [403]:
# connecting to the server 

from pymongo import MongoClient

client = MongoClient("mongodb://timanderin.info:27017")
db = client.speech_db
debates_collection = db.presidential_debates


In [370]:
#function to extract date from the url

def parse_date_from_url(url):
    # tokenize using '/' as seperator and select the last token
    date_token = url.strip('/').split('/')[-1]
    # tokenize using '-' and assign the first three tokens to month, day and year
    date_string = '-'.join(date_token.split('-')[0:3])
    
    return datetime.datetime.strptime(date_string, '%B-%d-%Y')

        
        

In [424]:
# function to scrape text

import hashlib 

def scrape_debate_text(debate_url):
    # get date from url
    debate_date = parse_date_from_url(debate_url)
    
    # visit site
    browser.visit(debate_url)
    
    # scrape text
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    text_body = soup.find_all('div', id='content-sm')
    
    # for i in range(0, len(results[0])):
    debate_text = ''
    for para in text_body[0].find_all('p', recursive='false'):
        debate_text += para.text + '\n'
    debate_hash = hashlib.md5(debate_text.encode())
    
    return (debate_date, debate_hash.hexdigest(), debate_text)
    

In [402]:
print(scrape_debate_text('https://www.debates.org/voter-education/debate-transcripts/september-26-2016-debate-transcript/'))

(datetime.datetime(2016, 9, 26, 0, 0), 'a8bba79dea8f1524e6afbf8fcfb2228b', '\nPresidential Debate at Hofstra University in Hempstead, New York\nSeptember 26, 2016\nPARTICIPANTS:\nFormer Secretary of State Hillary Clinton (D) and\nBusinessman Donald Trump (R)\nMODERATOR:\nLester Holt (NBC News)\nHOLT: Good evening from Hofstra University in Hempstead, New York. I’m Lester Holt, anchor of “NBC Nightly News.” I want to welcome you to the first presidential debate.\nThe participants tonight are Donald Trump and Hillary Clinton. This debate is sponsored by the Commission on Presidential Debates, a nonpartisan, nonprofit organization. The commission drafted tonight’s format, and the rules have been agreed to by the campaigns.\nThe 90-minute debate is divided into six segments, each 15 minutes long. We’ll explore three topic areas tonight: Achieving prosperity; America’s direction; and securing America. At the start of each segment, I will ask the same lead-off question to both candidates, an

In [425]:
# dumping data to the server

articles = results[0].find_all('a')
for article in articles:
    href_link = article['href']
    debate_url = base_url + href_link
    try: 
        (debate_date, debate_hash, debate_text) = scrape_debate_text(debate_url)
        debates_collection.replace_one(
            { "hash": debate_hash },
            {
                "year": debate_date,
                "hash": debate_hash,
                "text": debate_text,
                "tag": "debate"
            }, 
            upsert=True
        )
    except:
        print(href_link, " could not be scraped")


/voter-education/debate-transcripts/2008-debate-transcript/  could not be scraped
/voter-education/debate-transcripts/2008-debate-transcript-2/  could not be scraped
/voter-education/debate-transcripts/2000-debate-transcripts-translations/  could not be scraped


In [420]:
# trial for the first page click

href_link = articles[0]['href']
href_link

'/voter-education/debate-transcripts/september-26-2016-debate-transcript/'

In [276]:
browser.find_link_by_href(href_link).click()

In [ ]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')